In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re

In [2]:
url = 'https://sofifa.com/players?offset=0'

In [3]:
def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup

# Scratch Work

In [4]:
soup = soup_maker(url)

In [5]:
scraped_data = []
tbl = soup.find('table',{'class':'table table-hover persist-area'})
tbdy = tbl.find('tbody')
all_a = tbdy.find_all('a', {'rel':None})
player_details = {}
# players on every other index
for i, lnk in enumerate(all_a):
    if i % 2 == 0:
        player_details['short_name'] = lnk.text
        #player_details.update(get_player_details('http://sofifa.com' + lnk['href']))
        player_details['link'] = 'http://sofifa.com' + lnk['href']
        print(player_details)
        scraped_data.append(player_details)

{'short_name': 'L. Messi', 'link': 'http://sofifa.com/player/158023/lionel-messi/'}
{'short_name': 'Cristiano Ronaldo', 'link': 'http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/'}
{'short_name': 'Neymar Jr', 'link': 'http://sofifa.com/player/190871/neymar-da-silva-santos-jr/'}
{'short_name': 'De Gea', 'link': 'http://sofifa.com/player/193080/david-de-gea-quintana/'}
{'short_name': 'K. De Bruyne', 'link': 'http://sofifa.com/player/192985/kevin-de-bruyne/'}
{'short_name': 'E. Hazard', 'link': 'http://sofifa.com/player/183277/eden-hazard/'}
{'short_name': 'L. Modrić', 'link': 'http://sofifa.com/player/177003/luka-modric/'}
{'short_name': 'L. Suárez', 'link': 'http://sofifa.com/player/176580/luis-suarez/'}
{'short_name': 'H. Kane', 'link': 'http://sofifa.com/player/202126/harry-kane/'}
{'short_name': 'J. Oblak', 'link': 'http://sofifa.com/player/200389/jan-oblak/'}
{'short_name': 'R. Lewandowski', 'link': 'http://sofifa.com/player/188545/robert-lewandowski/'}
{'short_name': 'T. 

In [6]:
all_deets = {}
soup = soup_maker('http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/')
plyr_info = soup.find('div', {'class':'player'})

# find_plyr_info
plyr_dat = {} # prepare dictionary for player data
ply_inf = plyr_info.find('div', {'class':'meta'}) # grab section with player data
infos = ply_inf.text
data_str = infos[infos.index('Age') + 4:] # grab all data in str to right of Age like height and weight

# set variables
plyr_dat['pref_pos'] = ply_inf.find('span').text
plyr_dat['full_name'] = ply_inf.text[0:ply_inf.text.find(plyr_dat['pref_pos'])-2].strip()
plyr_dat['age'] = int(data_str[:2])
plyr_dat['height'] = data_str[data_str.index(')') + 2:].split(' ')[0].replace('\"','')
plyr_dat['weight'] = data_str[data_str.index(')') + 2:].split(' ')[1]
print(plyr_dat)

{'pref_pos': 'ST', 'full_name': 'C. Ronaldo dos Santos Aveiro (Cristiano Ronaldo dos Santos Aveiro)', 'age': 33, 'height': "6'2", 'weight': '183lbs'}


In [7]:
plyr_data = {}
soup = soup_maker('http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/')
plyr_stats = soup.find('div', {'class':'stats'})
plyr_val = plyr_stats.text[plyr_stats.text.find('€'):].split('\n')[0]
info = re.findall('\d+', plyr_stats.text)
plyr_data['rating'] = int(info[0])
plyr_data['potential'] = int(info[1])

if 'M' in plyr_val:
    plyr_data['value'] = int(plyr_val[1:plyr_val.index('M')])*1000000
elif 'K' in plyr_val:
    plyr_data['value'] = int(plyr_val[1:plyr_val.index('K')])*1000

plyr_data['wage'] = int(info[3])*1000

print(plyr_data)

{'rating': 94, 'potential': 94, 'value': 77000000, 'wage': 405000}


In [8]:
soup = soup_maker('http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/')
sp = soup.find('div', {'class':'teams'})
lnks = sp.find_all('li')
club = lnks[8].find('a').text
joined = lnks[12].text
print(club)
print(lnks[12].find('label', text='Joined').parent.contents[2])

Juventus
Jul 10, 2018


# Ok, now let's turn this work into functions

In [9]:
def gather_basic_info(soup):
    plyr_dat = {} # prepare dictionary for player data
    ply_inf = soup.find('div', {'class':'meta'}) # grab section with player data
    infos = ply_inf.text
    if 'Age' not in infos:
        return(plyr_dat)
    data_str = infos[infos.index('Age') + 4:] # grab all data in str to right of Age like height and weight

    # set variables
    plyr_dat['pref_pos'] = ply_inf.find('span').text
    plyr_dat['full_name'] = ply_inf.text[0:ply_inf.text.find(plyr_dat['pref_pos'])-2].strip()
    plyr_dat['age'] = int(data_str[:2])
    plyr_dat['height'] = data_str[data_str.index(')') + 2:].split(' ')[0].replace('\"','')
    plyr_dat['weight'] = data_str[data_str.index(')') + 2:].split(' ')[1]
    return(plyr_dat)

In [38]:
def gather_club_info(soup):
    # gets club and join data info
    plyr_data = {}
    lnks = soup.find_all('li')
    
    # the link list isn't the same for each player
    if lnks[8].find('a'):
        plyr_data['Club'] = lnks[8].find('a').text
    else:
        plyr_data['Club'] = lnks[7].find('a').text
        
    # assign data
    if len(lnks) < 13:
        plyr_data['Joined'] = None
        return(plyr_data)
    
    if lnks[12].find('label', text='Joined'):
        plyr_data['Joined'] = lnks[12].find('label', text='Joined').parent.contents[2]
    else:
        plyr_data['Joined'] = None
    
    return(plyr_data)

In [11]:
def gather_player_stats(soup):
    # parse stats section and determine rating, potential, value, and wage
    plyr_data = {}
    plyr_val = soup.text[soup.text.find('€'):].split('\n')[0]
    info = re.findall('\d+', soup.text)
    
    plyr_data['rating'] = int(info[0])
    plyr_data['potential'] = int(info[1])

    # check units of their value
    if 'M' in plyr_val:
        plyr_data['value'] = int(float(plyr_val[1:plyr_val.index('M')])*1000000)
    elif 'K' in plyr_val:
        plyr_data['value'] = int(float(plyr_val[1:plyr_val.index('K')])*1000)

    # wage is always given in thousands
    if len(info) > 4:
        plyr_data['wage'] = int(info[4])*1000
    else:
        plyr_data['wage'] = int(info[3])*1000
    
    return(plyr_data)

In [12]:
def get_player_details(soup):
    # take in player url and build dictionary of player info
    all_deets = {}
    soup = soup_maker(soup)
    
    # gather basic name, height, weight stats
    plyr_info = soup.find('div', {'class': 'player'})
    all_deets.update(gather_basic_info(plyr_info))
    
    # gather rating, value, wage, etc
    plyr_stats = soup.find('div', {'class': 'stats'})
    all_deets.update(gather_player_stats(plyr_stats))
    
    club_info = soup.find('div', {'class': 'teams'})
    all_deets.update(gather_club_info(club_info))
    
    return(all_deets)

# Now let's create our dataframe

In [36]:
def scrape_sofifa():
    # completes entire scraping process, can take minutes to run
    scraped_data = []
    page_indexes = [i*60 for i in range(100)]
    
    # iterate across 100 different player pages
    for page_index in page_indexes:
        print('Scraping page {}...'.format(page_index))
        url = 'https://sofifa.com/players?offset=' + str(page_index)
        soup = soup_maker(url)
        
        # gather all player details for this page
        tbl = soup.find('table',{'class':'table table-hover persist-area'})
        tbdy = tbl.find('tbody')
        all_a = tbdy.find_all('a', {'rel':None})
        
        # players are on every other index
        for i, lnk in enumerate(all_a):
            if i % 2 == 0:
                player_details = {}
                player_details['short_name'] = lnk.text
                player_details['link'] = 'http://sofifa.com' + lnk['href']
                player_details.update(get_player_details('http://sofifa.com' + lnk['href']))
                scraped_data.append(player_details)    
    
    return(scraped_data)

In [ ]:
import pandas as pd
scraped_data = scrape_sofifa()
df = pd.DataFrame(scraped_data)
df.head()
df.to_csv('scraped_sofifa.csv', index=False)

Scraping page 0...
Scraping page 60...
Scraping page 120...
Scraping page 180...
Scraping page 240...
Scraping page 300...
Scraping page 360...
Scraping page 420...
Scraping page 480...
Scraping page 540...
Scraping page 600...
Scraping page 660...
Scraping page 720...
Scraping page 780...
Scraping page 840...
Scraping page 900...
Scraping page 960...
Scraping page 1020...
Scraping page 1080...
Scraping page 1140...
Scraping page 1200...
Scraping page 1260...
Scraping page 1320...
Scraping page 1380...
Scraping page 1440...
Scraping page 1500...
Scraping page 1560...
Scraping page 1620...
Scraping page 1680...
Scraping page 1740...
Scraping page 1800...
Scraping page 1860...
Scraping page 1920...
Scraping page 1980...
Scraping page 2040...
Scraping page 2100...
Scraping page 2160...
Scraping page 2220...
Scraping page 2280...
Scraping page 2340...


In [33]:
df

,Club,Joined,age,full_name,height,link,potential,pref_pos,rating,short_name,value,wage,weight
0,FC Barcelona,"Jul 1, 2004",31,Lionel Messi (Lionel Andrés Messi Cuccittini),5'7,http://sofifa.com/player/158023/lionel-messi/,94,CF,94,L. Messi,110500000,565000,159lbs
1,Juventus,"Jul 10, 2018",33,C. Ronaldo dos Santos Aveiro (Cristiano Ronald...,6'2,http://sofifa.com/player/20801/c-ronaldo-dos-s...,94,ST,94,Cristiano Ronaldo,77000000,405000,183lbs
2,Paris Saint-Germain,"Aug 3, 2017",26,Neymar da Silva Santos Jr. (Neymar da Silva Sa...,5'9,http://sofifa.com/player/190871/neymar-da-silv...,92,LW,92,Neymar Jr,108000000,290000,150lbs
3,Manchester United,"Jul 1, 2011",27,David De Gea Quintana,6'4,http://sofifa.com/player/193080/david-de-gea-q...,93,GK,91,De Gea,72000000,260000,168lbs
4,Manchester City,"Aug 30, 2015",27,Kevin De Bruyne,5'11,http://sofifa.com/player/192985/kevin-de-bruyne/,92,CAM,91,K. De Bruyne,102000000,355000,154lbs
5,Chelsea,"Jul 1, 2012",27,Eden Hazard,5'8,http://sofifa.com/player/183277/eden-hazard/,91,LW,91,E. Hazard,93000000,340000,168lbs
6,Real Madrid,"Aug 1, 2012",32,Luka Modrić,5'8,http://sofifa.com/player/177003/luka-modric/,91,CM,91,L. Modrić,67000000,420000,146lbs
7,FC Barcelona,"Jul 11, 2014",31,Luis Suárez (Luis Alberto Suárez Díaz),6'0,http://sofifa.com/player/176580/luis-suarez/,91,ST,91,L. Suárez,80000000,455000,190lbs
8,Tottenham Hotspur,"Jul 1, 2010",24,Harry Kane,6'2,http://sofifa.com/player/202126/harry-kane/,92,ST,90,H. Kane,96500000,210000,196lbs
9,Atlético Madrid,"Jul 16, 2014",25,Jan Oblak,6'2,http://sofifa.com/player/200389/jan-oblak/,93,GK,90,J. Oblak,68000000,94000,192lbs
